# 6. Kaavatietomallin implementointi

ETL-työkalussa [kaavatietomallin](https://tietomallit.ymparisto.fi/kaavatiedot/v1.1/looginenmalli/uml/doc/) implementointi tapahtuu muuntamalla GeoPandas GeoDataFrame -muotoinen kaava-aineisto GeoJSON-formaattiin.

In [ ]:
# Käytettävien Python moduulien ja kehitettyjen funktioiden sisäänluku
import geopandas as gpd
from lib.pandas_to_geojson import dataToGeoJSON

In [ ]:
# Kaava-aineisto
data = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")

# Aineiston muuntaminen kaavatietomalliin ja JSON-dumpiksi
data_json = dataToGeoJSON(kaavadata=data, kaavatunnus_col="<insert column name here>")

# GeoJSON tallennus
with open(r"<insert filepath here>", "w") as outfile:
    outfile.write(data_json)